## Лабораторна робота №2: Беггінг

**Мета:** Ознайомлення із беггінгом, дослідження його ефективності
для задачі класифікації з різними слабкими класифікаторами.

In [30]:
# Імпорт необхідних бібліотек
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier, VotingClassifier
from sklearn.metrics import accuracy_score, precision_score, f1_score, classification_report
import warnings

# Ігнорувати попередження для чистоти виводу
warnings.filterwarnings('ignore', category=UserWarning)
warnings.filterwarnings('ignore', category=FutureWarning)

### Завантаження даних

In [31]:
# Завантажуємо датасет Titanic. Обов'язково вкажіть правильний шлях до файлу.
df = pd.read_csv('data/Titanic/Titanic-Dataset.csv')
print("Датасет успішно завантажено.")
print("Перші 5 рядків даних:")
df.head()

Датасет успішно завантажено.
Перші 5 рядків даних:


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


### Попередня обробка даних

In [32]:
# Визначення ознак та цільової змінної
# Визначаємо категоріальні та числові ознаки, які будемо використовувати.
# Цільова змінна - 'Survived'.
categorical_features = ['Sex', 'Embarked']
numerical_features = ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare']
target = 'Survived'

# Видаляємо непотрібні стовпці
# 'PassengerId', 'Name', 'Ticket' не несуть корисної інформації для моделі.
# 'Cabin' має занадто багато пропущених значень.
print("\nСтовпці перед видаленням:", df.columns)
df = df.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1)
print("Стовпці після видалення:", df.columns)


Стовпці перед видаленням: Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')
Стовпці після видалення: Index(['Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare',
       'Embarked'],
      dtype='object')


#### Розділення даних на навчальну та тестову вибірки

In [33]:
# Розділяємо дані перед будь-якою обробкою
X = df.drop(target, axis=1)
y = df[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(f"\nРозмір навчальної вибірки X: {X_train.shape}")
print(f"Розмір тестової вибірки X: {X_test.shape}")
print(f"Розмір навчальної вибірки y: {y_train.shape}")
print(f"Розмір тестової вибірки y: {y_test.shape}")
print(f"\nРозподіл класів у навчальній вибірці:\n{y_train.value_counts(normalize=True)}")
print(f"Розподіл класів у тестовій вибірці:\n{y_test.value_counts(normalize=True)}")


Розмір навчальної вибірки X: (712, 7)
Розмір тестової вибірки X: (179, 7)
Розмір навчальної вибірки y: (712,)
Розмір тестової вибірки y: (179,)

Розподіл класів у навчальній вибірці:
Survived
0    0.616573
1    0.383427
Name: proportion, dtype: float64
Розподіл класів у тестовій вибірці:
Survived
0    0.614525
1    0.385475
Name: proportion, dtype: float64


#### Створення конвеєрів для обробки ознак
Створюємо два конвеєри (pipelines): один для числових, інший для категоріальних ознак.

In [34]:
# Конвеєр для числових ознак:
# 1. SimpleImputer: Заповнює пропущені значення ('Age', 'Fare') медіаною.
# 2. StandardScaler: Масштабує ознаки до нульового середнього та одиничної дисперсії.
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# Конвеєр для категоріальних ознак:
# 1. SimpleImputer: Заповнює пропущені значення ('Embarked') найчастішим значенням (модою).
# 2. OneHotEncoder: Перетворює категоріальні ознаки на числові за допомогою one-hot encoding.
#    handle_unknown='ignore' ігнорує категорії, які можуть з'явитися в тестовій вибірці, але були відсутні в навчальній.
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

#### Об'єднання конвеєрів за допомогою ColumnTransformer

In [35]:
# ColumnTransformer застосовує відповідний конвеєр до відповідних стовпців.
# remainder='passthrough' залишає стовпці, не вказані в transformers, без змін (у нашому випадку таких немає після видалення)
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ],
    remainder='passthrough'
)

print("\nСтворено препроцесор для ознак.")


Створено препроцесор для ознак.


### Визначення слабких класифікаторів та їх ансамблів

#### Налаштування класифікаторів

In [36]:
# Використовуємо стандартні гіперпараметри
# random_state=42 для відтворюваності результатів.
# max_iter=1000 для MLPClassifier, щоб уникнути попереджень про незбіжність.
# probability=True для SVC, щоб мати можливість використовувати 'soft' voting в кінцевому ансамблі.

mlp_clf = MLPClassifier(random_state=42, max_iter=2000)
svm_clf = SVC(random_state=42, probability=True) 
dt_clf = DecisionTreeClassifier(random_state=42)

print("Слабкі класифікатори визначено: MLP, SVM, Decision Tree.")

Слабкі класифікатори визначено: MLP, SVM, Decision Tree.


#### Створення ансамблів за допомогою BaggingClassifier

In [37]:
# Використовуємо BaggingClassifier з параметрами за замовчуванням.
# bootstrap=True означає використання бутстрепу (resampling with replacement).
# n_estimators=10 - стандартна кількість базових моделей в ансамблі.

bagging_mlp = BaggingClassifier(
    # Важливо: передаємо новий екземпляр класифікатора в estimator
    estimator=MLPClassifier(random_state=42, max_iter=2000),
    n_estimators=10, random_state=42, n_jobs=-1 # n_jobs=-1 використовує всі доступні ядра процесора
)

bagging_svm = BaggingClassifier(
    estimator=SVC(random_state=42, probability=True),
    n_estimators=10, random_state=42, n_jobs=-1
)

bagging_dt = BaggingClassifier(
    estimator=DecisionTreeClassifier(random_state=42),
    n_estimators=10, random_state=42, n_jobs=-1
)

print("Ансамблі Bagging створено для MLP, SVM, Decision Tree.")

Ансамблі Bagging створено для MLP, SVM, Decision Tree.


#### Створення комбінованого ансамблю (слабкий класифікатор 1 + 2 + 3)
Використовуємо VotingClassifier для об'єднання прогнозів трьох *забеггованих* моделей.<br>voting='soft' використовує прогнозовані ймовірності, що часто дає кращі результати.<br>Використовуємо забегговані моделі як базові оцінювачі для цього фінального ансамблю.

In [38]:
# Створюємо нові екземпляри забегованих моделей для VotingClassifier,
# щоб уникнути потенційних проблем зі станом моделей після попереднього навчання 
bagged_mlp_for_voting = BaggingClassifier(estimator=MLPClassifier(random_state=42, max_iter=2000), n_estimators=10, random_state=42, n_jobs=-1)
bagged_svm_for_voting = BaggingClassifier(estimator=SVC(random_state=42, probability=True), n_estimators=10, random_state=42, n_jobs=-1)
bagged_dt_for_voting = BaggingClassifier(estimator=DecisionTreeClassifier(random_state=42), n_estimators=10, random_state=42, n_jobs=-1)


combined_ensemble = VotingClassifier(
    estimators=[
        ('bagged_mlp', bagged_mlp_for_voting),
        ('bagged_svm', bagged_svm_for_voting),
        ('bagged_dt', bagged_dt_for_voting)
    ],
    voting='soft' # Використовуємо м'яке голосування на основі ймовірностей
)

print("Комбінований ансамбль VotingClassifier створено.")

Комбінований ансамбль VotingClassifier створено.


### Навчання та оцінка моделей

#### Створення списку моделей для ітерації

In [39]:
# Зберігаємо моделі та їх назви для зручного навчання та оцінки в циклі.
# Важливо: ми використовуємо необроблені класифікатори тут. Обробка даних буде застосована через Pipeline.

models = {
    "слабкий класифікатор 1 (MLP)": mlp_clf,
    "ансамбль (слабкий класифікатор 1 - Bagged MLP)": bagging_mlp,
    "слабкий класифікатор 2 (SVM)": svm_clf,
    "ансамбль (слабкий класифікатор 2 - Bagged SVM)": bagging_svm,
    "слабкий класифікатор 3 (Decision Tree)": dt_clf,
    "ансамбль (слабкий класифікатор 3 - Bagged DT)": bagging_dt,
    "ансамбль (Bagged MLP + Bagged SVM + Bagged DT)": combined_ensemble
}

results = {} # Словник для зберігання результатів

#### Цикл навчання та оцінки

In [40]:
print("\n--- Початок навчання та оцінки моделей ---")
for name, model in models.items():
    print(f"\nНавчання моделі: {name}")

    # Створення повного конвеєра: препроцесор + модель
    # Важливо створювати пайплайн всередині циклу, щоб кожна модель навчалася з "чистим" препроцесором
    pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                               ('classifier', model)])

    # Навчання моделі
    try:
        pipeline.fit(X_train, y_train)

        # Прогнозування на тестовій вибірці
        y_pred = pipeline.predict(X_test)

        # Оцінка моделі
        accuracy = accuracy_score(y_test, y_pred)
        # Використовуємо weighted average для precision та f1-score, щоб врахувати можливий дисбаланс класів
        precision = precision_score(y_test, y_pred, average='weighted', zero_division=0)
        f1 = f1_score(y_test, y_pred, average='weighted', zero_division=0)

        # Збереження результатів
        # Метод бутстрапу для всіх ансамблів BaggingClassifier - 'resampling with replacement' (стандартний)
        # Для окремих класифікаторів та VotingClassifier бутстрап не застосовується напряму.
        bootstrap_method = "resampling with replacement" if isinstance(model, BaggingClassifier) else "N/A"
        if isinstance(model, VotingClassifier): # Спеціальний випадок для VotingClassifier
            bootstrap_method = "N/A (Voting Ensemble)"

        results[name] = {
            "Bootstrap": bootstrap_method,
            "Accuracy (test)": accuracy,
            "Precision (test)": precision,
            "F1 Score (test)": f1
        }

        print(f"Результати для {name}:")
        print(f"  Accuracy: {accuracy:.4f}")
        print(f"  Precision (weighted): {precision:.4f}")
        print(f"  F1 Score (weighted): {f1:.4f}")
        # print("\nClassification Report:")
        # print(classification_report(y_test, y_pred, zero_division=0)) # Детальний звіт

    except Exception as e:
        print(f"Помилка під час навчання або оцінки моделі {name}: {e}")
        results[name] = {
            "Bootstrap": "Error",
            "Accuracy (test)": np.nan,
            "Precision (test)": np.nan,
            "F1 Score (test)": np.nan
        }


print("\n--- Навчання та оцінка завершено ---")


--- Початок навчання та оцінки моделей ---

Навчання моделі: слабкий класифікатор 1 (MLP)
Результати для слабкий класифікатор 1 (MLP):
  Accuracy: 0.7877
  Precision (weighted): 0.7854
  F1 Score (weighted): 0.7849

Навчання моделі: ансамбль (слабкий класифікатор 1 - Bagged MLP)
Результати для ансамбль (слабкий класифікатор 1 - Bagged MLP):
  Accuracy: 0.7933
  Precision (weighted): 0.7917
  F1 Score (weighted): 0.7893

Навчання моделі: слабкий класифікатор 2 (SVM)
Результати для слабкий класифікатор 2 (SVM):
  Accuracy: 0.8156
  Precision (weighted): 0.8163
  F1 Score (weighted): 0.8112

Навчання моделі: ансамбль (слабкий класифікатор 2 - Bagged SVM)
Результати для ансамбль (слабкий класифікатор 2 - Bagged SVM):
  Accuracy: 0.8045
  Precision (weighted): 0.8075
  F1 Score (weighted): 0.7977

Навчання моделі: слабкий класифікатор 3 (Decision Tree)
Результати для слабкий класифікатор 3 (Decision Tree):
  Accuracy: 0.8156
  Precision (weighted): 0.8141
  F1 Score (weighted): 0.8136

Нав

### Виведення результатів у вигляді таблиці

In [48]:
print("\n--- Підсумкова таблиця результатів ---")
results_df = pd.DataFrame.from_dict(results, orient='index')
# Перейменування стовпців для відповідності таблиці з лабораторної
results_df = results_df.rename(columns={
    "Accuracy (test)": "Average Accuracy (test)",
    "Precision (test)": "Average Precision (test)", # В лабораторній це "(validation)", але ми рахували на test
    "F1 Score (test)": "F1 score (test)"
})

# Встановлення порядку стовпців як у лабораторній
column_order = [
    "Bootstrap",
    "Average Accuracy (test)",
    "Average Precision (test)",
    "F1 score (test)"
]
# Додаємо відсутні стовпці, якщо їх немає, і перевпорядковуємо
for col in column_order:
    if col not in results_df.columns:
        results_df[col] = "N/A" # Або інше значення за замовчуванням

results_df = results_df[column_order]


print(results_df.to_markdown(floatfmt=".4f")) # Вивід у форматі Markdown


--- Підсумкова таблиця результатів ---
|                                                | Bootstrap                   |   Average Accuracy (test) |   Average Precision (test) |   F1 score (test) |
|:-----------------------------------------------|:----------------------------|--------------------------:|---------------------------:|------------------:|
| слабкий класифікатор 1 (MLP)                   | N/A                         |                    0.7877 |                     0.7854 |            0.7849 |
| ансамбль (слабкий класифікатор 1 - Bagged MLP) | resampling with replacement |                    0.7933 |                     0.7917 |            0.7893 |
| слабкий класифікатор 2 (SVM)                   | N/A                         |                    0.8156 |                     0.8163 |            0.8112 |
| ансамбль (слабкий класифікатор 2 - Bagged SVM) | resampling with replacement |                    0.8045 |                     0.8075 |            0.7977 |
| слабкий кл